In [36]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import seaborn as sns
import numpy as np

In [37]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [38]:
# Read in faults_diagnostics data (1049678 rows)
faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv', index_col= 'EventTimeStamp', parse_dates = ['EventTimeStamp', 'LocationTimeStamp'])
faults_diagnostics = faults_diagnostics.drop_duplicates()
faults_diagnostics.shape

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\942447830.py:2: DtypeWarning: Columns (19,20,23,24,26,27,28,29,30,31,32,33,35,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv', index_col= 'EventTimeStamp', parse_dates = ['EventTimeStamp', 'LocationTimeStamp'])


(1054111, 42)

In [39]:
faults_diagnostics['PartialDerate'] = (faults_diagnostics['spn'] == 1569)
faults_diagnostics['FullDerate'] = (faults_diagnostics['spn'] == 5246)

rolling_7D = faults_diagnostics.loc[::-1].groupby('EquipmentID', as_index = False)[['PartialDerate', 'FullDerate']].rolling('7D').sum()
rolling_7D.loc[rolling_7D['PartialDerate'] > 0, 'PartialDerate'] = 1
rolling_7D.loc[rolling_7D['FullDerate'] > 0, 'FullDerate'] = 1

rolling_7D['PartialDerate'] = ((rolling_7D['EquipmentID'].shift() != rolling_7D['EquipmentID']) | (rolling_7D['PartialDerate'].shift() != rolling_7D['PartialDerate'])).cumsum()
rolling_7D['FullDerate'] = ((rolling_7D['EquipmentID'].shift() != rolling_7D['EquipmentID']) | (rolling_7D['FullDerate'].shift() != rolling_7D['FullDerate'])).cumsum()

faults_diagnostics = faults_diagnostics.drop(columns = ['PartialDerate', 'FullDerate'])
faults_diagnostics = faults_diagnostics.merge(rolling_7D, on = ['EventTimeStamp', 'EquipmentID'])
faults_diagnostics = faults_diagnostics.sort_values(['EquipmentID', 'EventTimeStamp']).drop_duplicates()

In [40]:
partial_derate_clusters = faults_diagnostics.loc[faults_diagnostics['PartialDerate'].isin(faults_diagnostics.loc[faults_diagnostics['spn'] == 1569, 'PartialDerate'])]
partial_derate_mask = partial_derate_clusters.groupby('EquipmentID')['spn'].apply(lambda x: x.shift().eq(1569).cumsum().eq(0)).to_list()
partial_derates = partial_derate_clusters[partial_derate_mask].reset_index()

partial_derates['DeltaT'] = partial_derates.loc[::-1].groupby(['EquipmentID', 'PartialDerate'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
partial_derates['DeltaT'] = partial_derates.loc[::-1].groupby(['EquipmentID', 'PartialDerate'])['DeltaT'].cumsum()[::-1]

In [41]:
partial_derates = partial_derates.loc[partial_derates['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'PartialDerate', 'spn'], keep = 'last')
partial_derates['DeltaT'] = partial_derates['DeltaT'].fillna(0)

partial_derates.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

,count,mean,std,min,25%,50%,75%,max
spn,,,,,,,,
1569,491.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3362,95.0,1.770570,5.530648,0.055556,0.999722,1.000000,1.000833,52.859722
111,94.0,35.240287,50.043673,0.026389,0.961458,6.269306,52.806458,164.119722
1761,70.0,7.867294,21.247722,0.000278,0.000278,0.124306,2.648750,110.734444
3251,54.0,15.923508,23.228611,0.000278,2.328403,4.492639,20.735764,123.701111
3216,48.0,20.974115,34.054952,0.002778,1.177500,5.112361,23.184583,149.280000
3226,45.0,29.574272,43.513908,0.210278,1.023889,9.063333,37.613889,164.225556
596,38.0,55.224444,44.688077,0.758611,17.968542,48.519861,96.655556,146.443056
641,35.0,15.581127,26.790030,0.195556,1.195417,2.885556,15.991111,101.368056


In [75]:
full_derate_clusters = faults_diagnostics.loc[faults_diagnostics['FullDerate'].isin(faults_diagnostics.loc[faults_diagnostics['spn'] == 5246, 'FullDerate'])]
full_derate_mask = full_derate_clusters.groupby('EquipmentID')['spn'].apply(lambda x: x.shift().eq(5246).cumsum().eq(0)).to_list()
full_derates = full_derate_clusters[full_derate_mask].reset_index()

full_derates['DeltaT'] = full_derates.loc[::-1].groupby(['EquipmentID', 'FullDerate'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
full_derates['DeltaT'] = full_derates.loc[::-1].groupby(['EquipmentID', 'FullDerate'])['DeltaT'].cumsum()[::-1]

full_derates = full_derates.loc[full_derates['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'FullDerate', 'spn'], keep = 'last')
full_derates['DeltaT'] = full_derates['DeltaT'].fillna(0)

full_derates.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

,count,mean,std,min,25%,50%,75%,max
spn,,,,,,,,
5246,191.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1569,151.0,12.849152,14.843403,1.018333,4.000556,6.621667,16.364306,99.539722
111,44.0,25.889766,41.801047,0.028056,1.786181,7.511528,26.580972,163.804444
3362,40.0,15.913681,19.943055,0.114722,3.848681,7.912917,19.109792,100.540000
1761,38.0,18.371323,28.780975,0.000278,4.469722,5.641389,20.062569,147.710278
3216,37.0,24.989797,20.078970,0.174722,11.695000,21.151111,32.138333,70.280000
3226,35.0,30.162119,34.548427,0.000278,5.863194,16.865833,32.587778,132.666111
3031,34.0,13.469739,15.828093,1.175556,4.998750,7.134306,16.715903,69.679167
3364,29.0,8.636073,7.692951,0.309722,4.587778,5.485278,11.817222,34.015278


In [43]:
faults_diagnostics.drop_duplicates().shape

(1054624, 44)

In [48]:
faults_diagnostics = faults_diagnostics.sort_index()

In [49]:

faults_diagnostics['EventCluster_7D'] = 1

rolling_window_7D = faults_diagnostics.groupby('EquipmentID', as_index = False)['EventCluster_7D'].rolling('7D').count()

rolling_window_7D['EventCluster_7D'] = (rolling_window_7D['EventCluster_7D'].shift() >= rolling_window_7D['EventCluster_7D']).cumsum() + 1


In [50]:
faults_diagnostics = faults_diagnostics.drop(columns = ['EventCluster_7D'])
faults_diagnostics = faults_diagnostics.merge(rolling_window_7D, on = ['EventTimeStamp', 'EquipmentID'])
faults_diagnostics = faults_diagnostics.drop_duplicates()

In [51]:
faults_diagnostics.sort_values(['EquipmentID', 'EventTimeStamp'])

,RecordID,ESS_Id,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,PartialDerate,FullDerate,EventCluster_7D
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-05-11 13:11:20,49415,2363162,Incorrect Data J1939 Network #1 Primary Vehicl...,unknown,unknown,unknown,unknown,11,639,2,True,127,301,105431712,36.189398,-82.795601,2015-05-11 13:11:57,49415,14.4,13.92,False,0.00000,120841.7,185.0,18.0,55.1,193.6062,1145.75,2372.75,76.4,17070.798,2.760607,134.6,True,154.4,1279,False,NaN,4.475814,3276.75,14.4,0.58,1,3,1
2015-05-13 08:22:32,51363,2400445,Condition Exists Cruise Control Enable Switch,unknown,unknown,unknown,unknown,49,596,31,True,3,301,105431712,35.872500,-84.475648,2015-05-13 08:23:09,51363,89.6,14.4275,False,0.00000,121687.1,186.8,38.0,52.78,217.175,1257.5,2388.15,72.4,17193.373832,6.326941,100.4,True,120.2,4351,False,NaN,64.63717,3276.75,89.6,4.64,1,3,1
2015-05-18 09:34:05,57330,2482983,Abnormal Rate of Change Aftertreatment 1 Outle...,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,3226,10,True,6,301,105431712,35.972870,-83.920555,2015-05-18 09:34:40,57330,0.0,14.355,False,0.00000,123057.1,185,0.0,55.68,213.9688,1230.25,2413.65,50.8,17376.312978378,0,120.2,True,118.4,17407,False,NaN,63.52549,3276.75,0.0,20.3,1,3,1
2015-05-21 13:57:35,61706,2554027,Incorrect Data J1939 Network #1 Primary Vehicl...,unknown,unknown,unknown,unknown,11,639,2,True,127,301,105431712,36.384953,-86.478379,2015-05-21 13:58:10,61706,12.0,14.5725,False,0.00000,124496.5,183.2,14.0,51.62,196.7,1115.5,2443.5,41.2,17577.34790995,2.311513,109.4,True,107.6,1279,False,NaN,4.364162,3276.75,12.0,1.45,1,3,2
2015-05-21 14:54:32,61801,2555325,Incorrect Data J1939 Network #1 Primary Vehicl...,unknown,unknown,unknown,unknown,11,639,2,False,127,301,105431712,36.384814,-86.478379,2015-05-21 14:54:27,61801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-06 14:14:13,1248457,123906113,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0.0,27.26,221.7312,863.25,586.75,23.6,4042.492826,0.0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100.0,1.74,3508,1611,192433
2020-03-06 14:15:34,1248458,123906131,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,False,8,2377,108605700,35.027314,-85.323472,2020-03-06 14:15:30,1248458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,3508,1611,192433
2020-03-04 10:06:10,1247712,123513793,Low (Severity Low) Catalyst Tank Level,05572391*03002399*110719010101*62602004*N1*BKR*,80239683,6X1u20D1500000000,CMMNS,0,1761,17,True,1,2380,108616776,35.491620,-86.434074,2020-03-04 10:06:47,1247712,0.0,14.2825,False,0.00000,511.711578,167.0,12.0,30.74,165.65,599.625,22.75,32.0,65.514669,0.607598,NaN,True,118.4,1023,True,NaN,0.0,13.95,100.0,0.29,3509,1612,192434


In [52]:
eventClusters = faults_diagnostics.reset_index()
eventClusters['DeltaT'] = eventClusters.loc[::-1].groupby(['EquipmentID', 'EventCluster_7D'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
eventClusters['DeltaT'] = eventClusters.loc[::-1].groupby(['EquipmentID', 'EventCluster_7D'])['DeltaT'].cumsum()[::-1]
eventClusters = eventClusters.loc[eventClusters['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'EventCluster_7D', 'spn'])
eventClusters['DeltaT'] = eventClusters['DeltaT'].fillna(0)

In [55]:
eventClusters.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

,count,mean,std,min,25%,50%,75%,max
spn,,,,,,,,
111,65985.0,9.150436,26.931894,0.000000,0.016111,0.199167,2.624167,167.999722
929,49213.0,3.185358,12.917126,0.000000,0.002778,0.172222,1.480556,167.987778
829,17902.0,22.412568,40.045457,0.000000,0.194722,2.218611,23.305069,167.995000
96,17498.0,22.559635,40.216258,0.000000,0.199514,2.241111,23.391111,167.995000
639,11817.0,26.843894,45.434924,0.000000,0.131944,1.494722,30.696667,167.998056
596,7966.0,27.786056,45.001537,0.000000,0.222639,3.288472,32.871875,167.999444
97,4957.0,16.997287,36.245528,0.000000,0.035278,0.793611,12.171111,167.988333
50353,3577.0,25.704551,42.379115,0.000000,0.048889,2.380833,32.780278,167.901944
1569,3036.0,19.081749,35.551129,0.000000,0.034097,2.117917,19.731667,167.988611


In [65]:
partial_d_cluster_list = partial_derates['PartialDerate'].unique().tolist()

frequencies = pd.DataFrame(columns=['spn', 'count', 'proportion'])

for i in partial_d_cluster_list:
    cluster = partial_derates[partial_derates['PartialDerate'] == i].value_counts('spn').reset_index().rename(columns={0:'count'})
    cluster['proportion'] = cluster['count'] / sum(cluster['count']) * 100
    frequencies = frequencies.append(cluster)

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

...

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppData\Local\Temp\ipykernel_26852\1969888555.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frequencies = frequencies.append(cluster)
C:\Users\Conrad\AppD

## Frequency calcs

In [113]:
frequency_all = pd.DataFrame(eventClusters.value_counts('spn')).reset_index().rename(columns = {0:"count"})
frequency_all = frequency_all[(frequency_all['spn'] != 1569)&(frequency_all['spn'] != 5246)]
frequency_all['proportion'] =( frequency_all['count']/sum(frequency_all['count'])) *100
frequency_all

,spn,count,proportion
0,111,65985,25.807146
1,929,49213,19.247512
2,829,17902,7.001584
3,96,17498,6.843577
4,639,11817,4.621702
5,596,7966,3.115552
6,97,4957,1.938714
7,50353,3577,1.398987
9,1761,2777,1.086102
10,3226,2578,1.008272


In [112]:
frequency_partial = pd.DataFrame(partial_derates.value_counts('spn')).reset_index().rename(columns = {0:"count"})
frequency_partial = frequency_partial[frequency_partial['spn'] != 1569]
frequency_partial['proportion'] =( frequency_partial['count']/sum(frequency_partial['count'])) *100
frequency_partial

,spn,count,proportion
1,3362,95,8.098892
2,111,94,8.013640
3,1761,70,5.967604
4,3251,54,4.603581
5,3216,48,4.092072
6,3226,45,3.836317
7,596,38,3.239557
8,641,35,2.983802
9,829,32,2.728048
10,102,31,2.642796


In [115]:
frequency_full = pd.DataFrame(full_derates.value_counts('spn')).reset_index().rename(columns = {0:"count"})
frequency_full = frequency_full[(frequency_full['spn'] != 1569)&(frequency_full['spn'] != 5246)]
frequency_full['proportion'] =( frequency_full['count']/sum(frequency_full['count'])) *100
frequency_full

,spn,count,proportion
2,111,44,7.131280
3,3362,40,6.482982
4,1761,38,6.158833
5,3216,37,5.996759
6,3226,35,5.672609
7,3031,34,5.510535
8,3364,29,4.700162
9,5394,19,3.079417
10,96,18,2.917342
11,829,18,2.917342
